# Programming Project - Unit 1,2
*by Igor A. Brandão*

**Goals**
- Access data from ckan.imd.ufrn.br
- Generate a Histogram/bar figure reporting the number of "municipal schools" and "health units"
- Geocoder figure about those units
- A heatmap figure considering the number of employees in each unit

## Imports

Import the necessary libraries to handle geocoding and maps

In [34]:
### Library necessary to run this IPython Notebook
!pip install geocoder
!pip install folium
!pip install tqdm

In [35]:
# Import tthe necessary libraries

import pandas as pd
import geocoder as gc
import numpy as np
import folium
from folium.plugins import HeatMap
from tqdm import tqdm
from bokeh.charts import Bar, output_notebook, show
from bokeh.layouts import row
from bokeh.models import HoverTool
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend

# Receive data from ckan
sab = "http://ckan.imd.ufrn.br/dataset/121bed36-e220-410c-8e87-eabfe2319bd7/resource/83c21ba5-c3a9-4876-8e65-3168ba392d1a/download/servicos-de-atencao-basica.csv"
sae = "http://ckan.imd.ufrn.br/dataset/121bed36-e220-410c-8e87-eabfe2319bd7/resource/52935484-0a23-4e13-b920-277448c66c8f/download/servicos-de-atencao-especializada.csv"

# Split the data using ';'
data1 = pd.read_csv(sae, sep=";")
data2 = pd.read_csv(sab, sep=";")

# Print the data
print(data1.columns.values.tolist())
print(data2.columns.values.tolist())

['Distrito Sanitário', 'Unidade', 'Serviços Oferecidos', 'Tipo de Unidade']
['Distrito Sanitário', 'Unidade', 'Serviços Oferecidos', 'Tipo de Unidade']


## Data printing

In [8]:
data1.head()

,Distrito Sanitário,Unidade,Serviços Oferecidos,Tipo de Unidade
0,Leste,CAPS AD II LESTE,Consulta Clinico Geral,CENTRO DE ATENÇÃO PSICOSSOCIAL
1,Leste,CAPS AD II LESTE,Consulta Enfermagem,CENTRO DE ATENÇÃO PSICOSSOCIAL
2,Leste,CAPS AD II LESTE,Consulta Nutrição,CENTRO DE ATENÇÃO PSICOSSOCIAL
3,Leste,CAPS AD II LESTE,Consulta de Psicologia,CENTRO DE ATENÇÃO PSICOSSOCIAL
4,Leste,CAPS AD II LESTE,Consulta Assistente Social,CENTRO DE ATENÇÃO PSICOSSOCIAL


In [9]:
data2.tail()

,Distrito Sanitário,Unidade,Serviços Oferecidos,Tipo de Unidade
815,Sul,USF PLANALTO,Consulta Odontologica,NaN
816,Sul,USF PLANALTO,Consulta Nutrição,NaN
817,Sul,USF PLANALTO,Pré-Natal,NaN
818,Sul,USF PLANALTO,CD Coletivo,NaN
819,Sul,USF PLANALTO,Regulação,NaN


## Data appending

Joing the data from different array in only one.

Add an index column and remove the previous one.

In [39]:
# Append data2 in data1
data1 = data1.append(data2)

# Reset the index
data1 = data1.reset_index()

# Remove the the previous index
data1.drop(data1.columns[0], axis=1, inplace=True)

# Print the new data1
data1

,Distrito Sanitário,Unidade,Serviços Oferecidos,Tipo de Unidade
0,Leste,CAPS AD II LESTE,Consulta Clinico Geral,CENTRO DE ATENÇÃO PSICOSSOCIAL
1,Leste,CAPS AD II LESTE,Consulta Enfermagem,CENTRO DE ATENÇÃO PSICOSSOCIAL
2,Leste,CAPS AD II LESTE,Consulta Nutrição,CENTRO DE ATENÇÃO PSICOSSOCIAL
3,Leste,CAPS AD II LESTE,Consulta de Psicologia,CENTRO DE ATENÇÃO PSICOSSOCIAL
4,Leste,CAPS AD II LESTE,Consulta Assistente Social,CENTRO DE ATENÇÃO PSICOSSOCIAL
5,Leste,CAPS AD II LESTE,Vivências lúdicas integrativas,CENTRO DE ATENÇÃO PSICOSSOCIAL
6,Leste,CAPS AD II LESTE,Atividade Física,CENTRO DE ATENÇÃO PSICOSSOCIAL
7,Leste,CAPS AD II LESTE,Psiquiatria,CENTRO DE ATENÇÃO PSICOSSOCIAL
8,Leste,CAPS AD II LESTE,Geriatria,CENTRO DE ATENÇÃO PSICOSSOCIAL
9,Leste,CAPS AD II LESTE,Internação em clínica médica,CENTRO DE ATENÇÃO PSICOSSOCIAL


# Health units handler

In [40]:
# Retrieve the uniques health units
unique_health_units = data1["Distrito Sanitário"].unique().tolist()

# Count the health units
units_count_dict = {}
for atual in unique_health_units:
    units_count_dict[atual] = data1[data1 == atual]["Distrito Sanitário"].count() 

# Read the data from dictionary
health_units = pd.DataFrame.from_dict(units_count_dict, orient='index')

# Get the data with index 'Region'
health_units.index.name = "Region"

# Append the count column
health_units.columns = ["Count"]

# Reset the data index
health_units = health_units.reset_index()

# Print the dataSet
print(health_units)


     Region  Count
0     Leste    687
1   Norte I    526
2  Norte II   1051
3     Oeste    710
4       Sul    509


## Health units chart

Histogram/bar figure reporting the number of "health units"

In [41]:
hover = HoverTool(tooltips=[('Quantidade', '$y{int}')])

bar = Bar(unidades_count, values="Count", label="Region", color="Region", title= "Unidades de saúde por região",
         ylabel ="Quantidade de unidades")
bar.add_tools(hover)

output_notebook()

show(bar)

Loading BokehJS ...

# Schools handler

In [42]:
# Receive data from ckan
url_schools = "http://ckan.imd.ufrn.br/dataset/9b362c15-832b-4aa9-9dfe-a5e015b3ce54/resource/8ad5f0d9-3ffc-41c4-84c4-f7e0cf4f2b5b/download/total-de-estabelecimentos-por-modalidade-de-ensino.csv"

# Split the data using ';'
data2 = pd.read_csv(url_schools, sep=";")

# Print the dataSet head
data2.head()

,Regiões Administrativas,Creches (CMEIS),Pré-escolas de Ens. Infantil (CMEIS),Pré-escolas de Ens. Infantil,Anos Iniciais do Ens. Fundamental,Anos Finais do Ens. Fundamental,EJA
0,SUL,11,11,2,7,5,3
1,LESTE,9,6,5,6,3,1
2,OESTE,21,20,4,22,8,11
3,NORTE,23,25,1,27,15,10


## Schools chart

Histogram/bar figure reporting the number of "municipal schools"

In [43]:
# Chart layers
BLEND = blend('Creches (CMEIS)', "Pré-escolas de Ens. Infantil (CMEIS)", "Pré-escolas de Ens. Infantil",
              "Anos Iniciais do Ens. Fundamental", "Anos Finais do Ens. Fundamental", 'EJA',
              name='unidade', labels_name='Unidades')

# Attach the dataSet and layers to the bar chart
bar = Bar(data2, values=BLEND,
          label=cat(columns='Regiões Administrativas', sort=False),
          stack=cat(columns='Unidades', sort=False),
          color="Unidades",
          legend='top_left',
          title="Total de escolas por região",
          ylabel = "Soma das unidades",
          tooltips=[('Tipo', '@Unidades')])

# Print the chart
output_notebook()
show(bar)

Loading BokehJS ...

# Geolocation handler

### Geolocation - schools

In [44]:
# Receive data from ckan
cmeis = "http://ckan.imd.ufrn.br/dataset/9b362c15-832b-4aa9-9dfe-a5e015b3ce54/resource/6d8e8580-bb48-4d75-a55c-4fa04da23919/download/cmeis-por-regioes-administrativas.csv"
schools = "http://ckan.imd.ufrn.br/dataset/9b362c15-832b-4aa9-9dfe-a5e015b3ce54/resource/99e0eef6-e16c-4ed8-bf62-d6bca9626eeb/download/escolas-por-regioes-administrativas.csv"

# Split the data using ';'
data3 = pd.read_csv(cmeis, sep=";")
data4 = pd.read_csv(schools, sep=";")

# Print the dataSet head
print(data3.head())
print(data4.head())

  Região Administrativa    CÓDIGO                       ESTABELECIMENTO  \
0                   SUL  24077720            CMEI CLAUDETE COSTA MACIEL   
1                   SUL  24077739  CMEI HAYDEE MONTEIRO BEZERRA DE MELO   
2                   SUL  24056936            CMEI KÁTIA FAGUNDES GARCIA   
3                   SUL  24077267              CMEI MARIA CELONI CAMPOS   
4                   SUL  24077500       CMEI MOEMA TINOCO DA CUNHA LIMA   

                     ENDEREÇO    Nº         BAIRRO       CEP        FONE  
0       RUA SERRA DOS CARAJAS  3160        PITIMBU  59068200  32328403.0  
1            RUA JOSÉ SELEDON    70    PONTA NEGRA  59090215  32328413.0  
2  RUA PROFESSORA ANA DJANIRA  1960     CANDELÁRIA  59064480  87291989.0  
3            RUA BAIA FORMOSÁ  1517  LAGOA NOVA II  59063060  32329443.0  
4                   RUA JACUI   217       NEÓPOLIS  59080270  32328376.0  
  Região Administrativa    CÓDIGO                        ESTABELECIMENTO  \
0                   SUL

In [45]:
# Append data4 in data3
data3 = data3.append(data4)

# Reset the index
data3 = data3.reset_index()

# Remove the the previous index
data3.drop(data3.columns[0], axis=1, inplace=True)

# Print the new data1
data3

,Região Administrativa,CÓDIGO,ESTABELECIMENTO,ENDEREÇO,Nº,BAIRRO,CEP,FONE
0,SUL,24077720,CMEI CLAUDETE COSTA MACIEL,RUA SERRA DOS CARAJAS,3160,PITIMBU,59068200,32328403.0
1,SUL,24077739,CMEI HAYDEE MONTEIRO BEZERRA DE MELO,RUA JOSÉ SELEDON,70,PONTA NEGRA,59090215,32328413.0
2,SUL,24056936,CMEI KÁTIA FAGUNDES GARCIA,RUA PROFESSORA ANA DJANIRA,1960,CANDELÁRIA,59064480,87291989.0
3,SUL,24077267,CMEI MARIA CELONI CAMPOS,RUA BAIA FORMOSÁ,1517,LAGOA NOVA II,59063060,32329443.0
4,SUL,24077500,CMEI MOEMA TINOCO DA CUNHA LIMA,RUA JACUI,217,NEÓPOLIS,59080270,32328376.0
5,SUL,24084298,CMEI PROFA ANTÔNIA FERNANDA JALLES,RUA RIO SUASSUI,7701,PITIMBU,59068320,NaN
6,SUL,24081221,CMEI PROFA CARMEM MARIA REIS,RUA JOÃO DE DEUS DE LIMA,SN,PONTA NEGRA,59090318,32324812.0
7,SUL,24064440,CMEI PROFA LIBÂNIA MEDEIROS,AV. SÃO MIGUEL DOS CARIBES,4334,NEÓPOLIS,59086500,88145388.0
8,SUL,24085332,CMEI PROFA Mª DOS MARTIRIOS LISBOA DE MENEZES,RUA ENGENHEIRO JOAQUIM CARDOSO,2020,PITIMBU,59069010,88877803.0
9,SUL,24064785,CMEI PROFA MARIA EUNICE DAVIM,RUA SOBRALIA,S/N,NEÓPOLIS,59088680,87063068.0


In [46]:
#data3["GeoCod"] = data3["ESTABELECIMENTO"] + ", " + data3["ENDEREÇO"] + ", "+ data3["BAIRRO"]
data3['lat'], data3['long'] = [0, 0]

In [47]:
data3.head()

,Região Administrativa,CÓDIGO,ESTABELECIMENTO,ENDEREÇO,Nº,BAIRRO,CEP,FONE,lat,long
0,SUL,24077720,CMEI CLAUDETE COSTA MACIEL,RUA SERRA DOS CARAJAS,3160,PITIMBU,59068200,32328403.0,0,0
1,SUL,24077739,CMEI HAYDEE MONTEIRO BEZERRA DE MELO,RUA JOSÉ SELEDON,70,PONTA NEGRA,59090215,32328413.0,0,0
2,SUL,24056936,CMEI KÁTIA FAGUNDES GARCIA,RUA PROFESSORA ANA DJANIRA,1960,CANDELÁRIA,59064480,87291989.0,0,0
3,SUL,24077267,CMEI MARIA CELONI CAMPOS,RUA BAIA FORMOSÁ,1517,LAGOA NOVA II,59063060,32329443.0,0,0
4,SUL,24077500,CMEI MOEMA TINOCO DA CUNHA LIMA,RUA JACUI,217,NEÓPOLIS,59080270,32328376.0,0,0


In [19]:
# Retrieve the latitude and longitute related to each school
for i in tqdm(range(len(data3))):
    g = gc.google(data3.loc[i,'ESTABELECIMENTO'])
    if g.lat == None:
        str = data3.loc[i,'ENDEREÇO']+", "+ data3.loc[i,'BAIRRO']+" - RN"
        g = gc.google(str)
    data3.ix[i,'lat'] = g.lat
    data3.ix[i,'long'] = g.lng
print('Geocoding complete!')

100%|██████████| 145/145 [02:46<00:00,  1.14s/it]

Geocoding complete!


In [20]:
data3.head()

,Região Administrativa,CÓDIGO,ESTABELECIMENTO,ENDEREÇO,Nº,BAIRRO,CEP,FONE,lat,long
0,SUL,24077720,CMEI CLAUDETE COSTA MACIEL,RUA SERRA DOS CARAJAS,3160,PITIMBU,59068200,32328403.0,-5.861867,-35.237338
1,SUL,24077739,CMEI HAYDEE MONTEIRO BEZERRA DE MELO,RUA JOSÉ SELEDON,70,PONTA NEGRA,59090215,32328413.0,-5.889422,-35.171142
2,SUL,24056936,CMEI KÁTIA FAGUNDES GARCIA,RUA PROFESSORA ANA DJANIRA,1960,CANDELÁRIA,59064480,87291989.0,-5.843191,-35.218472
3,SUL,24077267,CMEI MARIA CELONI CAMPOS,RUA BAIA FORMOSÁ,1517,LAGOA NOVA II,59063060,32329443.0,-5.820928,-35.224054
4,SUL,24077500,CMEI MOEMA TINOCO DA CUNHA LIMA,RUA JACUI,217,NEÓPOLIS,59080270,32328376.0,-5.858978,-35.206216


In [21]:
# Export the dataSet to csv
data3.to_csv('GeoCode_Escolas.csv', encoding="utf-8")

### Gelocation - health units

In [22]:
# Retrieve the health units geolocation
data1['lat'], data1['long'] = [0, 0]

for i in tqdm(range(len(data1))):
    g = gc.google(data1.loc[i,'Unidade'])
    data1.ix[i,'lat'] = g.lat
    data1.ix[i,'long'] = g.lng
print('Geocoding complete!')

100%|██████████| 1023/1023 [09:09<00:00,  1.94it/s]

Geocoding complete!


In [23]:
data1.head()

,Distrito Sanitário,Unidade,Serviços Oferecidos,Tipo de Unidade,lat,long
0,Leste,CAPS AD II LESTE,Consulta Clinico Geral,CENTRO DE ATENÇÃO PSICOSSOCIAL,-5.786882,-35.200245
1,Leste,CAPS AD II LESTE,Consulta Enfermagem,CENTRO DE ATENÇÃO PSICOSSOCIAL,-5.786882,-35.200245
2,Leste,CAPS AD II LESTE,Consulta Nutrição,CENTRO DE ATENÇÃO PSICOSSOCIAL,-5.786882,-35.200245
3,Leste,CAPS AD II LESTE,Consulta de Psicologia,CENTRO DE ATENÇÃO PSICOSSOCIAL,-5.786882,-35.200245
4,Leste,CAPS AD II LESTE,Consulta Assistente Social,CENTRO DE ATENÇÃO PSICOSSOCIAL,-5.786882,-35.200245


In [24]:
# Export the health unit dataSet to .csv
data1.to_csv('GeoCode_Saude.csv', encoding="utf-8")

In [25]:
# Read the generate csv
geodata1 = pd.read_csv('GeoCode_Escolas.csv', encoding="utf-8", index_col=0)
geodata2= pd.read_csv('GeoCode_Saude.csv', encoding="utf-8", index_col=0)

In [26]:
# Retrieve the data related to geolocalization
geodata = geodata1.filter(['ESTABELECIMENTO','lat','long'], axis=1)
geodata = geodata.rename(columns = {'ESTABELECIMENTO':'Unidade'})
geodata = geodata.append( geodata2.filter(['Unidade','lat','long'], axis=1))

# Reset the index
geodata = geodata.reset_index()

# Remove the the previous index
geodata.drop(geodata.columns[0], axis=1, inplace=True)
#geodata.head()

In [27]:
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Create map object
map_osm = folium.Map(location=mapc, zoom_start=zoom)

# Plot each of the locations that we geocoded
for j in tqdm(range(len(geodata))):
    folium.Marker([geodata.ix[j,'lat'], geodata.ix[j,'long']],
                  #popup=(geodata.ix[j,'Unidade'])
                 ).add_to(map_osm)
# Show the map
map_osm


100%|██████████| 1168/1168 [00:17<00:00, 67.10it/s]


In [28]:
# Retrieve the professionals by region
dLeste = "http://ckan.imd.ufrn.br/dataset/c434f105-bc9e-4415-aeb7-6ec54d6d0351/resource/c6480b62-4ade-4b80-a918-1c22dcef7c68/download/profissionais-das-unidades-do-distrito-leste.csv"
dNortI = "http://ckan.imd.ufrn.br/dataset/c434f105-bc9e-4415-aeb7-6ec54d6d0351/resource/f90b1a83-d17e-4ada-88d8-0dc8003634a9/download/profissionais-das-unidades-do-distrito-norte-i.csv"
dNorteII = "http://ckan.imd.ufrn.br/dataset/c434f105-bc9e-4415-aeb7-6ec54d6d0351/resource/f0ffb77f-9f4d-4f0f-82d3-089d2126789f/download/profissionais-das-unidades-do-distrito-norte-ii.csv"
dOeste = "http://ckan.imd.ufrn.br/dataset/c434f105-bc9e-4415-aeb7-6ec54d6d0351/resource/486f84b9-00ac-456b-bdd4-fad57af19cf9/download/profissionais-das-unidades-do-distrito-oeste.csv"
dSul = "http://ckan.imd.ufrn.br/dataset/c434f105-bc9e-4415-aeb7-6ec54d6d0351/resource/73902e96-f86f-4ab9-9887-c38b2c1f3d60/download/profissionais-das-unidades-do-distrito-sul.csv"

# Split the data using ';'
dataH = pd.read_csv(dLeste, sep=";")
dataH2 = pd.read_csv(dNortI, sep=";")
dataH3 = pd.read_csv(dNorteII, sep=";")
dataH4 = pd.read_csv(dOeste, sep=";")
dataH5 = pd.read_csv(dSul, sep=";")

# Append all dataSet in only one variable
dataH = dataH.append(dataH2).append(dataH3).append(dataH4).append(dataH5)

# Print the dataSet head
dataH.head()

,UNIDADE,NOME DO PROFISSIONAL,ESPECIALIDADE,HORÁRIO
0,APARECIDA,PATRICIA ARBOES PETRONILO,MÉDICO,07:30 ÁS 11:30 / 14:00 ÁS 17:00
1,APARECIDA,ANA LILIAN PROTASIO DE LIMA,MÉDICO,07:30 ÁS 11:30 / 14:00 ÁS 17:00
2,APARECIDA,NEUMA MARINHO DE QUEIROZ SANTOS,MÉDICO,07:30 ÁS 11:30 / 14:00 ÁS 17:00
3,APARECIDA,MARIA CRISTINA VARELA BARCA,ENFERMEIRO,07:30 ÁS 11:30 / 14:00 ÁS 17:00
4,APARECIDA,MAGNA MARIA PEREIRA DA SILVA,ENFERMEIRO,07:30 ÁS 11:30 / 14:00 ÁS 17:00


In [29]:
# Do some fix in final data
dataFinal = dataH.copy()
dataFinal.drop(dataFinal.columns[-3:], axis=1, inplace=True)
dataFinal["Count"] = 0

# Print the dataSet head
dataFinal.head()

,UNIDADE,Count
0,APARECIDA,0
1,APARECIDA,0
2,APARECIDA,0
3,APARECIDA,0
4,APARECIDA,0


In [30]:
# Cound the number of units by neighborhood
dataFinal = pd.DataFrame(dataFinal.groupby(["UNIDADE"])['Count'].count()).reset_index()

# Print the dataSet head
dataFinal.head()

,UNIDADE,Count
0,APARECIDA,31
1,BAIRRO NORDESTE,39
2,BOM PASTOR,37
3,BRASÍLIA TEIMOSA,17
4,CIDADE NOVA,40


In [31]:
# Retrieve the latitude and longitude
dataFinal['lat'], dataFinal['long'] = [0, 0]

for i in tqdm(range(len(dataFinal))):
    g = gc.google("Unidade Saude"+dataFinal.loc[i,'UNIDADE']+" RN")
    dataFinal.ix[i,'lat'] = g.lat
    dataFinal.ix[i,'long'] = g.lng
print('Geocoding complete!')
dataFinal

100%|██████████| 39/39 [00:40<00:00,  1.05s/it]

Geocoding complete!


,UNIDADE,Count,lat,long
0,APARECIDA,31,-5.792801,-35.187280
1,BAIRRO NORDESTE,39,-5.796688,-35.242233
2,BOM PASTOR,37,-5.809690,-35.235417
3,BRASÍLIA TEIMOSA,17,-5.776584,-35.197312
4,CIDADE NOVA,40,-5.858202,-35.241673
5,CIDADE PRAIA,22,-5.739238,-35.267214
6,FELIPE CAMARÃO I,41,-5.829223,-35.249099
7,FELIPE CAMARÃO II,45,-5.815283,-35.249299
8,FELIPE CAMARÃO III,61,-5.824543,-35.264234
9,GRAMORÉ,34,-5.728282,-35.253950


## HeatMap

In [32]:
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Initialize the coordinates array
coordinates = []

# Add the coordinates to the coordinate
for i in range(len(dataFinal)):
    # eliminate items with'nan' element
    if all(~np.isnan([dataFinal.ix[i,'lat'], dataFinal.ix[i,'long'], dataFinal.ix[i,'Count']])):
        coordinates.append([dataFinal.ix[i,'lat'], dataFinal.ix[i,'long'], dataFinal.ix[i,'Count']])

# Create map object
htMap = folium.Map(location=mapc, zoom_start=zoom)

# Append the coordinates to the heatMap
HeatMap(coordinates).add_to(htMap)

# Print the heatMap
htMap